In [ ]:
#| default_exp helpers.submit

# Notebook Submission Helper (submit.py)

> This helper supports submitting completed notebooks for credit by extracting student responses and sending them to a remote grading service.

Answer format convention

Student answers are expected to appear in code cells using the following pattern:
```python
q1_answer = "A"
q2_answer = 3.14
q3_answer = ["token", "vector", "embedding"]
```
Where:
- Each answer variable name begins with q
- The question number follows (q1, q2, …)
- The variable is assigned a valid Python literal

This convention allows answers to be reliably extracted from the notebook without requiring special widgets or forms.

Submission workflow

The submission process has three steps:

1. Collect answers
Code cells matching the answer pattern are located and extracted.

2. Parse answers
Extracted code is converted into a structured Python dictionary.

3. Submit answers
The parsed answers are sent via HTTP to a grading service.

Each step is implemented as a separate function to keep the logic clear and testable.

In [ ]:
#| export
import os
import httpx
import nbformat
from pathlib import Path
import inspect
import re
import json

In [ ]:
#| export
def _can_use_dialoghelper():
    try:
        from dialoghelper import find_msgs
        return True
    except Exception:
        return False

In [ ]:
#| export
def _collect_answers_from_notebook(path=None):
    """
    Collect qN_answer assignments by parsing a notebook file directly.
    Works in local Jupyter.
    """
    if path is None:
        # Infer the current notebook path (best-effort)
        frame = inspect.currentframe()
        while frame:
            fname = frame.f_globals.get("__file__")
            if fname and fname.endswith(".ipynb"):
                path = fname
                break
            frame = frame.f_back

        if path is None:
            raise RuntimeError(
                "Could not infer notebook path. "
                "Pass path= explicitly when running locally."
            )

    nb = nbformat.read(Path(path), as_version=4)

    results = []
    pat = re.compile(r'^q\d+_answer\s*=', re.MULTILINE)

    for cell in nb.cells:
        if cell.cell_type == "code":
            for line in cell.source.splitlines():
                if pat.match(line.strip()):
                    results.append(("code", line.strip()))

    return results


In [ ]:
#| export
def collect_answers(show=True, *, dname=None, path=None):
    """
    Collect student answers from a notebook.

    - Uses dialoghelper when available (Solveit / Deepnote)
    - Falls back to parsing the notebook file locally (JupyterLab)
    
    Requires:
    - SUBMIT_API_KEY (loaded via helpers.env.load_env)

    Parameters
    ----------
    show : bool
        Print extracted answers
    dname : str, optional
        Dialog name (Solveit only)
    path : str or Path, optional
        Notebook path (required for reliable local execution)

    Returns
    -------
    list[tuple]
    """
    # Attempt dialoghelper first
    try:
        from dialoghelper import find_msgs

        msgs = find_msgs(
            re_pattern=r'^q\d+_answer\s*=',
            msg_type='code',
            dname=dname
        )
        results = [('code', msg['content'].strip()) for msg in msgs]

    except Exception:
        # Fallback: local notebook parsing
        results = _collect_answers_from_notebook(path=path)

    if show:
        print("=== Student Responses ===\n")
        for _, answer in results:
            print(answer)

    return results

In [ ]:
#| export
def parse_answers(raw):
    """
    Parse extracted answer code into a dictionary.

    Parameters
    ----------
    raw : list of tuple
        Output from `collect_answers`, containing raw code strings.

    Returns
    -------
    dict
        Mapping from answer variable name (e.g. 'q1_answer') to its value.
    """
    result = {}
    for typ, content in raw:
        if typ == 'code':
            # Parse "q1_answer = "A"" format
            key, val = content.split(' = ', 1)
            result[key] = eval(val)
    return result

In [ ]:
#| export
def submit_answers(
    student_id, 
    *, 
    path,
    raw_answers=None,
    dname=None, 
    url="https://nbsubmit-production.up.railway.app", 
    api_key=None,
    verbose=True,
):
    """
    Submit parsed student answers to the grading service.

    Parameters
    ----------
    student_id : str
        Identifier for the student submitting the notebook.
    raw_answers : list of tuple, optional
        Raw extracted answers. If omitted, answers are collected automatically.
    url : str
        Base URL of the submission service.
    api_key : str, optional
        API key for authentication. If not provided, read from environment.

    Returns
    -------
    dict
        Parsed JSON response from the submission server.
    """
    if not student_id.strip():
        raise ValueError("student_id cannot be empty")
    if raw_answers is None:
        raw_answers = collect_answers(show=False, dname=dname, path=path)
    answers = parse_answers(raw_answers)

    payload = {
        "student_id": student_id,
        "answers": answers,
    }

    if verbose:
        print("\n📤 YOUR SUBMISSION")
        print("=" * 60)
        print(json.dumps(payload, indent=2))
        print("=" * 60)
        
    headers = {"x-api-key": api_key or os.environ.get("SUBMIT_API_KEY")}
    key = api_key or os.environ.get("SUBMIT_API_KEY")
    if not key:
        raise RuntimeError(
            "Missing SUBMIT_API_KEY. Set it in your environment or .env file."
        )
    headers = {"x-api-key": key}
    response = httpx.post(f"{url}/submit", json={"student_id": student_id, "answers": answers}, headers=headers)
    return response.json()

In [ ]:
# Test (not exported)

from data401_nlp.helpers.submit import parse_answers

In [ ]:
#| export
def review_answers(*, path, dname=None, show=True):
    """
    Review extracted student answers WITHOUT submitting.

    Parameters
    ----------
    path : str or Path
        Path to the notebook being reviewed.
    dname : str, optional
        Dialog name (Solveit / Deepnote).
    show : bool
        Print formatted output.

    Returns
    -------
    dict
        Parsed answers ready for submission.
    """
    raw = collect_answers(show=False, dname=dname, path=path)
    answers = parse_answers(raw)

    if show:
        print("\n📝 REVIEW: Answers detected in this notebook")
        print("=" * 60)
        for k, v in answers.items():
            print(f"{k}: {v!r}")
        print("=" * 60)
        print("⚠️  Nothing has been submitted.")

    return answers


In [ ]:
raw = [
    ("code", 'q1_answer = "A"'),
    ("code", "q2_answer = 42"),
]

assert parse_answers(raw) == {
    "q1_answer": "A",
    "q2_answer": 42,
}

print("parse_answers OK")


parse_answers OK
